In [46]:
%load_ext autoreload
%autoreload 2

# guidelines

TODO : import whenever needed, not centralized

states https://pytorch.org/tutorials/beginner/saving_loading_models.html

# Introduction 

## Aim

## Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#%cd /content/drive/MyDrive/Colab\ Notebooks/
#%cd CS439/optml_project/
!ls

First load the dataset:

In [47]:
from data_utils import get_mnist

train_dataset, test_dataset = get_mnist(normalize=True)

In [48]:
import numpy as np
#import random
import torch
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Setup

Below one can find flags that will setup the notebook:

In [49]:
# Whether to tune the hyperparameters in this notebook
# Note that this might take a long time (especially for Adam)
hyperparameter_tune = False
prot_hyperparameter_tune = False

In [50]:
# Whether to use the GPU, if it's not available, this will be ignored
use_cuda = True
device = torch.device('cuda' if use_cuda and torch.cuda.is_available() else 'cpu')
print("Device chosen is {}".format(device))

Device chosen is cpu


We setup the training parameters that we will use all along the notebook, in order to improve readability in downstream code:

In [51]:
from training import accuracy

training_config = {
    # Loss function
    'loss_fun': torch.nn.CrossEntropyLoss(),
    # Performance evaluation function
    'metric_fun': accuracy,
    # The device to train on
    'device': device,
    # Number of epochs
    'epochs': 10,
}

test_config = training_config.copy()
test_config.pop('epochs');

Note that we will use a model with a 10-dimensional output, where each output is passed through softmax. When receiving an output 

$$Z = \begin{bmatrix} \mathbf z_1 & \dots & \mathbf z_B \end{bmatrix}^\top \in \mathbb R^{B \times 10}$$

with $B$ the batch size, we first retrieve the maximal component of each $\mathbf z_i$:

$$\hat y_i = \text{argmax}_{k = 1, \ldots, 10} \; z_{ik}, \quad i = 1, \ldots, B$$

and then compute the accuracy:

$$\text{acc} = \frac 1 B \sum_{i=1}^B I\left\{ \hat y_i = y_i \right\} $$

with $I$ the indicator function and $y_i \in \{1, \ldots, 10\}$ the true target. 

In [24]:
# View the source code
??accuracy

Signature: accuracy(yhat, y)
Docstring: <no docstring>
Source:   
def accuracy(yhat, y):
    prediction = yhat.argmax(dim=1)
    return (y.eq(prediction)).to(float).mean().item()
File:      ~/Documents/EPFL/CS439/optml_project/training.py
Type:      function


# Model

We use a simple standard model for the MNIST dataset (can be found [here](https://github.com/floydhub/mnist/blob/master/ConvNet.py)).

In [52]:
from net import Net

??Net

Init signature: Net()
Source:        
class Net(nn.Module):
    """ConvNet -> Max_Pool -> RELU -> ConvNet -> Max_Pool -> RELU -> FC -> RELU -> FC -> SOFTMAX"""
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
File:           ~/Documents/EPFL/CS439/optml_project/net.py
Type:           type
Subclasses:     


# Hyperparameter tuning

In [53]:
from torch.optim import Optimizer
from training import tune_optimizer
from optimizer import AdamOptimizer, NesterovOptimizer, MiniBatchOptimizer
from data_utils import get_best_hyperparams

If the `hyperparameter_tune` flag was set to `True` above, the following code will run hyperparameter tuning on all optimizers. Note that one can either run KFold cross validation (by providing `n_folds`) or use a simple train/test split (by providing `train_ratio`).

If the flag is set to `False`, the cell below will simply set up the hyperparameters that we carefully cross-validated:

In [54]:
optimizers = {
    AdamOptimizer: get_best_hyperparams('./res/adam_tuning_round3.json'),
    NesterovOptimizer: get_best_hyperparams('./res/nesterov_tuning_round2.json'),
    MiniBatchOptimizer: get_best_hyperparams('./res/minibatch_tuning_round2.json')
}

## Adam

In [55]:
search_grid_adam = {
        'lr': np.linspace(0.001, 0.01, 3),
        'beta1':  np.linspace(0.1, 0.9, 2),
        'beta2': np.linspace(0.5, 0.999, 2),
        'batch_size': [32, 64, 128],
        'weight_decay': np.linspace(0.001, 0.1, 2),
        'epsilon': np.linspace(1e-10, 1e-8, 2),
    }

if hyperparameter_tune:
    results_adam = tune_optimizer(
        model=Net().to(device),
        optim_fun=AdamOptimizer,
        xtrain=train_dataset.data,
        ytrain=train_dataset.targets,
        search_grid=search_grid_adam,
        nfolds=3,
        **training_config)

else:
    results_adam = optimizers[AdamOptimizer]

## Nesterov

In [ ]:
search_grid_nesterov = {
    'lr': np.logspace(0, 1),
    'batch_size': [32, 64, 128]
}

if hyperparameter_tune:
    results_nesterov = tune_optimizer(
        model=Net().to(device),
        optim_fun=NesterovOptimizer,
        xtrain=train_dataset.data,
        ytrain=train_dataset.targets,
        search_grid=search_grid_nesterov,
        nfolds=3,
        **training_config
    )

else:
    results_nesterov = optimizers[NesterovOptimizer]

## Minibatch

In [ ]:
search_grid_mini  = {
    'lr': np.linspace(0.00001, 0.01, 5),
    'batch_size': [32, 64, 128],
    'decreasing_lr': [0, 1],
}
if hyperparameter_tune:
    results_mini = tune_optimizer(
        model=Net().to(device),
        optim_fun=MiniBatchOptimizer,
        xtrain=train_dataset.data,
        ytrain=train_dataset.targets,
        search_grid=search_grid_mini,
        nfolds=3,
        **training_config
    )

else:
    results_mini = optimizers[MiniBatchOptimizer]

In [ ]:
print("ADAM: Highest Test Accuracy {:.4f} with standart deviation of {:.4f}".format(results_adam["metric_test"], results_adam["metric_test_std"]))
print("Hyperparameter set: Learning rate =  {:.4f}, Beta1 = {:.1f}, Beta2 = {:.3f}, Weight decay = {:.2f}, Epsilon = {:.8f},  Batch Size = {:.0f}\n".format(results_adam["lr"], results_adam["beta1"], results_adam["beta2"], results_adam["weight_decay"], results_adam["epsilon"], results_adam['batch_size']))
print("NESTEROV: Highest Test Accuracy {:.4f} with standart deviation of {:.4f}".format(results_nesterov["metric_test"], results_nesterov["metric_test_std"]))
print("Hyperparameter set: Learning rate =  {:.4f}, Batch Size = {:.0f}\n".format(results_nesterov["lr"], results_nesterov["batch_size"]))
print("MINIBATCH: Highest Test Accuracy {:.4f} with standart deviation of {:.4f}".format(results_mini["metric_test"], results_mini["metric_test_std"]))
print("Hyperparameter set: Learning rate =  {:.4f}, Decreasing Learning rate {:.1f}, Batch Size = {:.0f}\n".format(results_mini["lr"], results_mini["decreasing_lr"], results_mini["batch_size"]))


# Attack on naive model



In [ ]:
from data_utils import build_data_loaders
from training import training, testing
from adversary import attack, projected_attack

In [37]:
optimizers = {
    AdamOptimizer: get_best_hyperparams('./res/adam_tuning_round3.json', get_performance=False),
    NesterovOptimizer: get_best_hyperparams('./res/nesterov_tuning_round2.json', get_performance=False),
    MiniBatchOptimizer: get_best_hyperparams('./res/minibatch_tuning_round2.json', get_performance=False)
}

In [ ]:
optimizers

In [ ]:
# Logging data structures
data_naive = list()
data_naive_attack = list()

# Training / attack config
n_try = 3
batch_log_interval = -1
epsilons = np.arange(0, 0.6, 0.05)

for optimizer, optimizer_params in optimizers.items():
    print(f'--- {optimizer.__name__}')
    # --------- SETUP OPTIMIZER
    optimizer_params = optimizer_params.copy()
    # Instantiate data loaders with selected batch size
    batch_size = int(optimizer_params.pop('batch_size'))
    train_loader, test_loader = build_data_loaders(train_dataset, test_dataset, batch_size)
    
    # --------- Train & Attack several times per optimizer
    for n in range(1, n_try + 1):
        net = Net().to(device)
        optimizer_instance = optimizer(net.parameters(), **optimizer_params)
        # --------- TRAIN MODEL
        loss_train, acc_train = training(
            model=net, 
            dataset=train_loader, 
            optim=optimizer_instance,
            batch_log_interval=batch_log_interval,
            **training_config
        )
        # --------- TEST MODEL
        loss_test, acc_test = testing(
            model=net,
            dataset=test_loader,
            **test_config
        )
        # Log
        data_naive.append({
            'optimizer': str(optimizer_instance),
            'n': n,
            'loss_train': loss_train,
            'acc_train': acc_train,
            'loss_test': loss_test,
            'acc_test': acc_test
        })

        # --------- ATTACK MODEL
        print(f'Launching attacks', end=' ')
        for eps in epsilons:
            print('', end='.')
            loss_attack, acc_attack = attack(
                model=net,
                test_loader=test_loader,
                epsilon=eps,
                verbose=False,
                **test_config
            )
            # Log
            data_naive_attack.append({
                'optimizer': str(optimizer_instance),
                'n': n,
                'epsilon': eps,
                'loss': loss_attack,
                'acc': acc_attack
            })

        print()

### Training curves

In [ ]:
df_naive = pd.DataFrame(data_naive).sort_values(['optimizer', 'n'])
# Average training loss per epoch
df_naive.loss_train = df_naive.loss_train.apply(lambda s: np.mean(s, axis=1))

colors = {'AdamOptimizer': 'r', 'MiniBatchOptimizer': 'b', 'NesterovOptimizer': 'm'}
for _, row in df_naive.iterrows():
    plt.plot(range(1, training_config['epochs'] + 1), row.loss_train, '-o', label=row.optimizer, color=colors[row.optimizer])

plt.grid(alpha=.6)
plt.legend();
plt.yscale('log')
plt.xlabel('Epoch'); plt.ylabel('Training Loss')
plt.title('Training curves of naive models');

### Attack plots

In [ ]:
df = pd.DataFrame(data_naive_attack).sort_values(['optimizer', 'epsilon'])
# Raw accuracy in function of attack stength
sns.lineplot(x='epsilon', y='acc', data=df, hue='optimizer', marker='o')
plt.grid(alpha=.6)
plt.ylabel('Accuracy');
plt.title('Attack on naive models');

In [ ]:
df.T.to_json('res/log_attack_naive.json', indent=2)

# Attack on robust model

## Hyperparameter optimization on robust models

- If the `prot_hyperparameter_tune` flag was set to `True` above, the following code will run hyperparameter tuning on all optimizers for robust models. Note that one can either run KFold cross validation (by providing `n_folds`) or use a simple train/test split (by providing `train_ratio`).


In [16]:
from adversary import protected_training

In [62]:
prot_optimizers = {
    AdamOptimizer : get_best_hyperparams('./res/prot_res_adam_new1.json', get_performance=False),
    NesterovOptimizer: get_best_hyperparams('./res/prot_res_nesterov_new2.json', get_performance=False),
    MiniBatchOptimizer: get_best_hyperparams('./res/prot_res_minibatch_new2.json', get_performance=False)
}

### Adam

In [32]:
search_grid_adam = {
        'lr': np.linspace(0.001, 0.01, 3),
        'beta1':  np.linspace(0.1, 0.9, 2),
        'beta2': np.linspace(0.5, 0.999, 2),
        'batch_size': [32, 64, 128],
        'weight_decay': np.linspace(0.001, 0.1, 2),
        'epsilon': np.linspace(1e-10, 1e-8, 2),
    }

if prot_hyperparameter_tune:
    results_adam_prot = tune_optimizer(
        model=Net().to(device),
        optim_fun=AdamOptimizer,
        xtrain=train_dataset.data,
        ytrain=train_dataset.targets,
        search_grid=search_grid_adam,
        nfolds=3,
        func=protected_training,
        **training_config)

else:
    results_adam_prot = prot_optimizers[AdamOptimizer]

In [33]:
prot_optimizers[AdamOptimizer]

{'lr': 0.001,
 'beta1': 0.9,
 'beta2': 0.999,
 'weight_decay': 0.001,
 'epsilon': 1e-10,
 'batch_size': 128}

### Nesterov

In [44]:
search_grid_nesterov = {
    'lr': np.logspace(0, 1, num=15),
    'batch_size': [32, 64, 128]
}

if prot_hyperparameter_tune:
    results_nesterov_prot = tune_optimizer(
        model=Net().to(device),
        optim_fun=NesterovOptimizer,
        xtrain=train_dataset.data,
        ytrain=train_dataset.targets,
        search_grid=search_grid_nesterov,
        nfolds=3,
        func=protected_training,
        **training_config
    )

else:
    results_nesterov_prot = optimizers[NesterovOptimizer]

In [45]:
prot_optimizers[NesterovOptimizer]

{'lr': 2.6826957952797274e-05, 'batch_size': 128}

### Minibatch

In [ ]:
search_grid_mini  = {
        'lr': np.logspace(0,1,num=15),
        #'lr': np.linspace(0.00001, 0.01, 5),
        'batch_size': [32, 64, 128],
        'decreasing_lr': [0, 1],
    }
if prot_hyperparameter_tune:
    results_minibatch_prot = tune_optimizer(
        model=Net().to(device),
        optim_fun=MiniBatchOptimizer,
        xtrain=train_dataset.data,
        ytrain=train_dataset.targets,
        search_grid=search_grid_mini,
        nfolds=3,
        func=protected_training,
        **training_config
    )

else:
    results_minibatch_prot = optimizers[MiniBatchOptimizer]

In [ ]:
with open('./res/prot_res_minibatch_new1.json', 'w', encoding ='utf8') as json_file:
    json.dump(results_minibatch_prot, json_file, indent=2)

## Train & Attack robust models

In [58]:
from adversary import protected_training

In [60]:
# Logging data structures
data_robust = list()
data_robust_attack = list()

# Training / attack config
n_try = 3
batch_log_interval = -1
epsilons = np.arange(0, 0.6, 0.05)

for optimizer, optimizer_params in prot_optimizers.items():
    opt
    print(f'--- {optimizer.__name__}')
    # --------- SETUP OPTIMIZER
    optimizer_params = optimizer_params.copy()
    # Instantiate data loaders with selected batch size
    batch_size = int(optimizer_params.pop('batch_size'))
    train_loader, test_loader = build_data_loaders(train_dataset, test_dataset, batch_size)

    # --------- Train & Attack several times per optimizer
    for n in range(1, n_try + 1):
        net = Net().to(device)
        optimizer_instance = optimizer(net.parameters(), **optimizer_params)
        # --------- TRAIN MODEL
        loss_train, acc_train = protected_training(
            model=net,
            dataset=train_loader,
            optim=optimizer_instance,
            batch_log_interval=batch_log_interval,
            **training_config
        )
        # --------- TEST MODEL
        loss_test, acc_test = testing(
            model=net,
            dataset=test_loader,
            **test_config
        )
        # Log
        data_robust.append({
            'optimizer': str(optimizer_instance),
            'n': n,
            'loss_train': loss_train,
            'acc_train': acc_train,
            'loss_test': loss_test,
            'acc_test': acc_test
        })

        # --------- ATTACK MODEL
        print(f'Launching attacks', end=' ')
        for eps in epsilons:
            print('', end='.')
            loss_attack, acc_attack = attack(
                model=net,
                test_loader=test_loader,
                epsilon=eps,
                verbose=False,
                **test_config
            )
            # Log
            data_robust_attack.append({
                'optimizer': str(optimizer_instance),
                'n': n,
                'epsilon': eps,
                'loss': loss_attack,
                'acc': acc_attack
            })

        print()

TypeError: 'dict_items' object is not subscriptable

### Training curves

In [ ]:
df_naive = pd.DataFrame(data_robust).sort_values(['optimizer', 'n'])
# Average training loss per epoch
df_naive.loss_train = df_naive.loss_train.apply(lambda s: np.mean(s, axis=1))

colors = {'AdamOptimizer': 'r', 'MiniBatchOptimizer': 'b', 'NesterovOptimizer': 'm'}
for _, row in df_naive.iterrows():
    plt.plot(range(1, training_config['epochs'] + 1), row.loss_train, '-o', label=row.optimizer, color=colors[row.optimizer])

plt.grid(alpha=.6)
plt.legend();
plt.yscale('log')
plt.xlabel('Epoch'); plt.ylabel('Training Loss')
plt.title('Training curves of naive models');

### Attack plots

In [ ]:
df_robust = pd.DataFrame(data_robust_attack).sort_values(['optimizer', 'epsilon'])
# Raw accuracy in function of attack stength
sns.lineplot(x='epsilon', y='acc', data=df, hue='optimizer', marker='o')
plt.grid(alpha=.6)
plt.ylabel('Accuracy');
plt.title('Attack on robust models');

In [ ]:
df_robust.T.to_json('res/log_attack_robust.json', indent=2)

# Comparative analysis

In [ ]:
%%capture --no-stderr
# Whether to generate fancy plots for the report. Warning: this lengthens image rendering time
# This requires the `ipypublish` library
fancy_plots = True

if fancy_plots:
    from ipypublish import nb_setup
    plt = nb_setup.setup_matplotlib()
    # Override with seaborn defaults
    sns.set(style='whitegrid')

In [ ]:
from pathlib import Path

fig_path = Path('fig')
if not fig_path.exists():
    fig_path.mkdir()

In [ ]:
df_naive = pd.read_json('res/log_attack_naive.json').T.drop(columns=['loss', 'n'])
df_robust = pd.read_json('res/log_attack_robust.json').T
# For some reason this has to be done after parsing from json
df_naive.epsilon = df_naive.epsilon.astype(float)
df_naive.acc = df_naive.acc.astype(float)
df_robust.acc = df_robust.acc.astype(float)
df_robust.epsilon = df_robust.epsilon.astype(float)

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(10, 4), sharex=True)

plt.axes(ax[0])
sns.lineplot(x='epsilon', y='acc', data=df_naive, hue='optimizer', marker='o', ci='sd')
plt.yscale('log')
plt.ylabel('Accuracy');
plt.title('Attack on naive models');
plt.xlabel('Attack strength $\epsilon$')

plt.axes(ax[1])
sns.lineplot(x='epsilon', y='acc', data=df_robust, hue='optimizer', marker='o', ci='sd')
plt.grid(alpha=.6)
plt.ylabel('Accuracy');
plt.title('Attack on robust models')
plt.xlabel('Attack strength $\epsilon$')
plt.tight_layout();
sns.despine()
plt.savefig('fig/attacks.pdf', bbox_inches='tight')

In [ ]:
df_naive[df_naive.epsilon==0]

In [ ]:
df_robust[df_robust.epsilon == 0]

In [ ]:
df_robust